# 0.0. Imports

In [1]:
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn


from xgboost import XGBClassifier

from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp

# 1.0. Data Description

## 1.1. Loading Data

In [28]:
df_raw = pd.read_csv('data/test.csv')
df_country = pd.read_csv('data/country_base.csv')

In [29]:
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

## 1.2. Rename Columns

In [30]:
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

ValueError: Length mismatch: Expected axis has 17 elements, new values have 14 elements

## 1.3. Data Dimensions

In [ ]:
print('Number of rows: {}'.format(df_raw.shape[0]))
print('Number of cols: {}'.format(df_raw.shape[1]))

## 1.4. Check NA

In [35]:
df1.isna().sum()

id                                         0
Classificação do hotel                     0
Meses da reserva até o check-in            0
Número de pernoites reservadas             0
Número de hospedes                         1
Regime de alimentação                      0
Nacionalidade                            677
Forma de Reserva                           0
Já se hospedou anterioremente              0
Tipo do quarto reservado                   0
Reserva feita por agência de turismo       0
Reserva feita por empresa                  0
Reserva com Estacionamento                 0
Reserva com Observações                    0
country                                 1074
latitude                                1074
longitude                               1074
dtype: int64

### 1.4.1. Replace NA

## 1.5. Data Dtypes

In [7]:
df_raw.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
dtype: object

### 1.5.1. Change Dtypes

In [8]:
df_raw['numero_de_hospedes'] = df_raw['numero_de_hospedes'].astype('int')

## 1.6. Data Balancing

In [9]:
#df_raw['reserva_cancelada'].value_counts(normalize=True)

# 2.0. Feature Engineering

In [10]:
df2 = df_raw.copy()

# 3.0. Data Filtering

In [11]:
df3 = df2.copy()

# 4.0. Data Preparation 

In [12]:
#df4 = df3.drop('nacionalidade', axis=1).copy()
df4 = df3.copy()

In [13]:
# # dummy variables
# df4_dummy = pd.get_dummies(df4.drop(['id', 'meses_da_reserva_ate_o_check_in', 'numero_de_pernoites_reservadas',
#                                     'numero_de_hospedes'], axis=1))

# df4 = pd.concat([df4[['id', 'meses_da_reserva_ate_o_check_in', 'numero_de_pernoites_reservadas',
#                       'numero_de_hospedes']], df4_dummy], axis=1)

ohe = pickle.load(open('ohe.pkl', 'rb'))

# numerical
col_num = df4.select_dtypes(include=['int64', 'float64']).columns.tolist()

# categorical
col_cat = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']).columns.tolist()

# encoding
df4_dummy = pd.DataFrame(ohe.fit_transform(df4[col_cat]).toarray(), index=df4.index)

# join numerical and categorical
df42 = pd.concat([df4[col_num], df4_dummy], axis=1)
df42.columns = df42.columns.astype(str)


# 5.0. Feature Selection 

In [14]:
df5 = df42.drop('id', axis=1).copy()

In [15]:
# # split dataset into features and target
# X = df5.drop('reserva_cancelada', axis=1)
# y = df5['reserva_cancelada']

In [16]:
# # split dataset into train and test
# X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

## 1.5. Data Dtypes

# 6.0. Machine Learning Model

In [17]:
model = pickle.load(open('gradient_boosting.pkl', 'rb'))

In [18]:
yhat = model.predict(df5)

ValueError: Feature shape mismatch, expected: 202, got 189

# 7.0. Export CSV

In [ ]:
df2['Reserva Cancelada'] = yhat

In [ ]:
submission = df2[['id', 'Reserva Cancelada']].copy()

In [ ]:
submission.to_csv('submission.csv', index=False)

# 8.0. Model